In [1]:
import pathlib
from dataset import Dataset

Load the dataset

In [2]:
df = Dataset(csv_file_path=pathlib.Path('data_2023_07_20.csv'))

Dataset(45 entries)


Use the `.sub` method to filter entries and get a subset of the original dataset

In [7]:
when_at_home = df.sub(incl_act={'home'})
when_at_home

Dataset(15 entries)

In [9]:
bad_mood = df.sub(mood={1, 2, 3})
bad_mood

Dataset(16 entries)

In [10]:
cycling_df = df.sub(incl_act={'cycling'})
cycling_df

Dataset(11 entries)

Use the `.head` method to look at the first entries of the dataset

In [11]:
cycling_df.head()

Dataset(11 entries)
[2023-07-20 18:29] 4 shopping, cycling, unsure, nice weather
[2023-07-18 20:30] 5 tired, city, cycling, swimming, satisfied, friends, relaxed
[2023-07-18 14:29] 4 shopping, satisfied, cycling, nice weather
[2023-07-16 20:02] 4 cleaning, study, chess, cycling, home, dota, nail biting, friends
[2023-07-14 19:56] 5 study, tired, procrastinating, city, new place, cycling, swimming, friends, relaxed
...


Use the `.avg_mood` method to get the average mood of all the entries in the dataset

In [14]:
round(
    cycling_df.avg_mood(),
    3
)

4.273

Use the `.count` method to count entries with the given conditions.

This is equivalent to `len(df.sub(...))` with the same arguments, but the `.count` method does not create a new dataset.

In [8]:
df.count(incl_act={'friends'}, mood={2, 3})

2

Use the `.activities` method to get a Counter object of all activities in the dataset

In [9]:
df.activities().most_common(3)

[('nail biting', 16), ('home', 15), ('coding', 15)]

Use the `.get_datetimes` method to get the list of all points in time when an entry was created

In [34]:
df.get_datetimes()[:5]

[datetime.datetime(2023, 7, 20, 21, 21),
 datetime.datetime(2023, 7, 20, 18, 29),
 datetime.datetime(2023, 7, 20, 16, 58),
 datetime.datetime(2023, 7, 20, 13, 12),
 datetime.datetime(2023, 7, 19, 23, 10)]

## Analysis examples